In [ ]:
import os
from pathlib import Path

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Energytool Building

The aim of the tutorial is to introduce the energytool <code>Building</code> class.
It is build on top of [energyplus](https://energyplus.net/) and of the python library [eppy](https://github.com/santoshphilip/eppy).
It has been designed to simplify the HVAC, Domestic Hot Water (DHW) production and local energy production (Photovoltaic). It inherits from [corrai](https://github.com/BuildingEnergySimulationTools/corrai) <code>Model</code> base class, so it can be integrated to batch simulation workflows such as Sensitivity Analsysis, callibration, etc. 

- It has only one positional argument "idf_path". When instantiated it uses eppy.modeleditor <code>IDF</code> to read the idf file
- A <code>dict</code> is used to organise the HVAC DHW and production equipments objects. Its keys are <code>energytool.system SystemCategories</code> keys :
    HEATING, COOLING, VENTILATION, LIGHTING, AUXILIARY, DHW, PV, SENSOR, OTHER.

- <code>property</code> are defined to compute specific results.

Simulation can be run using <code>simulate()</code> method. The concept behind Energytool, is to simplified HVAC system modeling by editing the idf file before the simulation, and post-processing the energyplus results. Each <code> System </code> objects have a <code>pre_process</code> and a <code>post_process</code> method: 

1- <code>pre_process()</code> method of each system is called. Be careful, order of called objects matters in some cases. These methods modify the idf objects (create OutputVariables, modify power density, or materiel properties)

2- Energyplus is called and the simulation runs.

3- The <code>post_process()</code> method is called. Each objects apply post-processing to raw energyplus results and gather the results in a single Dataframe.

## 1- Instantiate a Building

Let's load an example idf file that model a 2 storey and 4 apartments building:

|             Figure 1: Building model perspective              |                 Figure 2: Building first floor                  |
|:-------------------------------------------------------------:|:---------------------------------------------------------------:|
| <img height="300" src="resources/building_tuto_3d_pers.png"/> | <img src="resources/building_tuto_1st_floor.png"  height="300"> |

In [ ]:
from energytool.building import Building

Building.set_idd(Path(r"C:\EnergyPlusV9-4-0"))

In [ ]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_building.idf")

<code>Building</code> have a <code>__repr__</code> method that displays basics information on the model

In [ ]:
building

Building surface, zone number, etc. are calculated from the imported idf file.
So far the system dict is empty.

However, it doesn't mean that no HVAC system are  modeled in the idf file.
 - Heating and ventilation are modeled using IdealLoadsAirSystem
 - Artificial Lighting is defined with the object Lights

__*The <code>Building</code> class allows you to add "Systems" that will edit a part of the  idf file and/or select and apply a post treatment to the simulation results. It cannot replace proper idf HVAC configuration. The use of IdealLoadsAirSystem is recommended.*__


## 2- Add systems

In energytool, a system is a class that modify an idf file or post process results.
The module <code>energytool.system</code> holds several examples of system class.
Custom class can be written, they must inherit from <code>energytool.system.System</code> base class:

- Every system have a <code>pre-process()</code> and a <code>post-process()</code> methods
- The other attributes depends on the system (COP, list of zone names, etc.)



In the following lines we add most of the systems present in <code>energytool.system</code> module.
 For more information on the effects of each system on the simulation, see the <code>energytool.system</code> documentation.

In [ ]:
from energytool.system import (
    HeaterSimple,
    HeatingAuxiliary,
    AirHandlingUnit,
    AHUControl,
    DHWIdealExternal,
    ArtificialLighting,
)

In [ ]:
# Simulate a boiler, multiplying the heat needs by a constant COP
building.add_system(HeaterSimple(name="Gaz_boiler", cop=0.89))

# Estimate circulation pumps energy consumption multiplying the heat needs by a constant (default 0.05)
building.add_system(HeatingAuxiliary(name="Heater_aux"))

# Simulate fan consumption multiplying extracted air volume by a constant coefficient
# Do not have a heat exchanger
building.add_system(
    AirHandlingUnit(
        name="Extraction_fan",
        fan_energy_coefficient=0.23,
        heat_recovery_efficiency=False,
    )
)

# Simulate clock regulation
# Ventilation works according to specified schedule
#  is defined in the energytool/resources/resources_idf.idf file
building.add_system(
    AHUControl(
        name="Hygro_intakes",
        control_strategy="Schedule",
        schedule_name="OFF_09h_18h_ON_18h_24h_ON_WE_FULL_YEAR",
    )
)

# Estimate Domestic Hot Water production energy needs
# Use the number of people defined in the idf file to estimate the total volume.
# Otherwise, energy calculation is independent of energyplus
building.add_system(
    DHWIdealExternal(
        name="Electric_accumulation",
        cop=0.85,
    )
)

# Estimate Lighting consumption using a constant power ratio.
# Modify the existing energyplus object
building.add_system(ArtificialLighting(name="Random_lights", power_ratio=4))

Building <code>__repr__</code> method can again be called

In [ ]:
building

## 4. Building simulation

### 4.1 Running the first simulation

In [ ]:
import datetime as dt
from energytool.building import SimuOpt

Energyplus doesn't care about the year. But it can be useful if you are calibrating a simulation, or if you need the right day of week (monda, tuesday, etc.). That's why the notion of "year" have been added to energytool. If you don't need it, you can leave <code>start</code> and <code>stop</code> to default. It will run the simulation for a full year.

Allowed simulation options are defined in <code>energytooL.building.SimuOpt</code> Enum. You can use its values.

In [ ]:
print([key.value for key in SimuOpt])

In [ ]:
res = building.simulate(
    parameter_dict=None,
    simulation_options={
        "epw_file": Path(TUTORIAL_DIR) / "resources/FRA_Bordeaux.075100_IWEC.epw",
        "start": "2025-01-01",
        "stop": "2025-12-31",
        "timestep": 15 * 60,  # seconds
        "outputs": "SYSTEM",  # See values in energytool.outputs.OutputCategories
    },
)

The output of the simulation is a DtaFrame. The <code>"outputs"</code> key in <code>simulation_options</code> defines what kind of result you need.
Lets plot the results in a pretty way:

In [ ]:
res[[col for col in res if col != "TOTAL_SYSTEM_Energy_[J]"]].sum().plot(kind="pie")

### 4.1 Modifying parameters


The main purpose of energytool is to be able to easily specify new values for parameters, wether it concerns idf properties, or energytool systems attributes.
To do so, A dictionary of parameter values can be passed to <code>simulate</code> method.
- The parameters names must be the "path" to the value.
 - For idf related objects, parameter name shall start with **idf**, for system attribute, it must start with **system**.
  - <code>epw_file</code> key, can be used in both parameters dictionary and simulation options. In some case, there is an interest to see climate as a parameter and not a boundary condition. An error will be raised if specified in both simulation options and parameters dictionary.  


In [ ]:
res_2 = building.simulate(
    parameter_dict={
        "idf.material.Urea Formaldehyde Foam_.1327.Conductivity": 0.05,
        "system.heating.Gaz_boiler.cop": 0.5,
    },
    simulation_options={
        "epw_file": Path(TUTORIAL_DIR) / "resources/FRA_Bordeaux.075100_IWEC.epw",
        "start": "2025-01-01",
        "stop": "2025-12-31",
        "timestep": 15 * 60,  # seconds
        "outputs": "SYSTEM",  # See values in energytool.outputs.OutputCategories
    },
)

In [ ]:
res_2[[col for col in res_2 if col != "TOTAL_SYSTEM_Energy_[J]"]].sum().plot(kind="pie")